- tested to be working on 3/5/19 Erica's laptop (with ocassional unknown server error occuring but fixed itself after running again)

In [3]:
# **OLD CODE, IGNORE
# setting the current working directory to the folder containing the PDF files
#os.chdir(r'C:\Users\erica\OneDrive\Data_projects\research_project_2018\pdfs')
#os.chdir(r'C:\Users\erica\Documents\pdfs_epicenter')

#pdf_pths = [pth for pth in Path.cwd().iterdir() if pth.suffix == '.pdf']
#pdf_pths = [str(file) for file in pdf_pths]

In [2]:
import os
from pathlib import Path

In [3]:
# Load Snorkel
%load_ext autoreload
%autoreload 2
%matplotlib inline

from snorkel import SnorkelSession
session = SnorkelSession()

#n_docs = 500 if 'CI' in os.environ else 2591

In [11]:
# Loading the corpus of documents

from snorkel.parser import TextDocPreprocessor
n_docs = 500
doc_preprocessor = TextDocPreprocessor('pdf_output.txt')

In [12]:
from snorkel.parser.spacy_parser import Spacy
from snorkel.parser import CorpusParser
from snorkel.models import Document, Sentence

corpus_parser = CorpusParser(parser=Spacy())
%time corpus_parser.apply(doc_preprocessor, count=n_docs)

Clearing existing...
Running UDF...





  0%|                                                                                          | 0/500 [00:00<?, ?it/s]


  0%|▏                                                                             | 1/500 [07:45<64:32:50, 465.67s/it]

Wall time: 7min 45s


In [13]:
print("Documents:", session.query(Document).count())
print("Sentences:", session.query(Sentence).count())

Documents: 1
Sentences: 26311


In [8]:
# Extract Candidates:
# First import classes to define candidates and their subclasses

# potential code:
#from snorkel.models import Candidate, candidate_subclass
#VirusHost = candidate_subclass('VirusHost', ['virus', 'host'])

### Writing a basic `CandidateExtractor`

Next, we'll write a basic function to extract **candidate spouse relation mentions** from the corpus.  The [Spacy](https://spacy.io/) parser we used performs _named entity recognition_ for us.

We will extract `Candidate` objects of the `Spouse` type by identifying, for each `Sentence`, all pairs of n-grams (up to 7-grams) that were tagged as people. (An n-gram is a span of text made up of n tokens.) We do this with three objects:

* A `ContextSpace` defines the "space" of all candidates we even potentially consider; in this case we use the `Ngrams` subclass, and look for all n-grams up to 7 words long

* A `Matcher` heuristically filters the candidates we use.  In this case, we just use a pre-defined matcher which looks for all n-grams tagged by Spacy as "PERSON". The keyword argument `longest_match_only` means that we'll skip n-grams contained in other n-grams.

* A `CandidateExtractor` combines this all together!

In [14]:
from snorkel.models import candidate_subclass

Colleagues = candidate_subclass('Colleagues', ['person1', 'person2'])

****TEST CODE ONLY

In [15]:
# To do next:
# Extract Candidates using a pretagged candidate extractor, eg TaggerOne NER


from snorkel.candidates import Ngrams, CandidateExtractor
from snorkel.matchers import PersonMatcher

ngrams         = Ngrams(n_max=7)
person_matcher = PersonMatcher(longest_match_only=True)
cand_extractor = CandidateExtractor(Colleagues, [ngrams, ngrams], [person_matcher, person_matcher])


In [16]:
from snorkel.models import Document
from util import number_of_people

docs = session.query(Document).order_by(Document.name).all()

train_sents = set()
dev_sents   = set()
test_sents  = set()

for i, doc in enumerate(docs):
    for s in doc.sentences:
        if number_of_people(s) <= 5:
            if i % 10 == 8:
                dev_sents.add(s)
            elif i % 10 == 9:
                test_sents.add(s)
            else:
                train_sents.add(s)

In [17]:
%%time
for i, sents in enumerate([train_sents]):
    cand_extractor.apply(sents, split=i)
    print("Number of candidates:", session.query(Colleagues).filter(Colleagues.split == i).count())

Clearing existing...
Running UDF...





  0%|                                                                                        | 0/26010 [00:00<?, ?it/s]


  0%|                                                                              | 27/26010 [00:00<01:36, 268.05it/s]


  0%|                                                                              | 36/26010 [00:00<02:46, 156.22it/s]


  0%|▏                                                                             | 50/26010 [00:00<02:52, 150.13it/s]


  0%|▏                                                                             | 60/26010 [00:00<03:27, 125.26it/s]


  0%|▎                                                                            | 101/26010 [00:00<02:44, 157.54it/s]


  1%|▍                                                                            | 137/26010 [00:00<02:16, 189.30it/s]


  1%|▌                                                                            | 179/26010 [00:00<01:54, 226.01it/s]


  1%|▌               

  9%|███████▏                                                                    | 2452/26010 [00:07<01:12, 325.25it/s]


 10%|███████▎                                                                    | 2487/26010 [00:07<01:17, 301.65it/s]


 10%|███████▎                                                                    | 2519/26010 [00:07<01:27, 267.92it/s]


 10%|███████▍                                                                    | 2556/26010 [00:07<01:20, 290.91it/s]


 10%|███████▌                                                                    | 2608/26010 [00:08<01:10, 334.10it/s]


 10%|███████▋                                                                    | 2650/26010 [00:08<01:06, 353.52it/s]


 10%|███████▉                                                                    | 2696/26010 [00:08<01:01, 379.21it/s]


 11%|███████▉                                                                    | 2737/26010 [00:08<01:04, 360.66it/s]


 11%|████████           

 18%|█████████████▊                                                              | 4725/26010 [00:15<01:22, 257.55it/s]


 18%|█████████████▉                                                              | 4768/26010 [00:15<01:12, 292.34it/s]


 18%|██████████████                                                              | 4800/26010 [00:15<01:13, 290.53it/s]


 19%|██████████████▏                                                             | 4843/26010 [00:15<01:06, 319.18it/s]


 19%|██████████████▎                                                             | 4877/26010 [00:15<01:08, 310.32it/s]


 19%|██████████████▎                                                             | 4910/26010 [00:15<01:07, 313.67it/s]


 19%|██████████████▍                                                             | 4943/26010 [00:16<01:12, 292.47it/s]


 19%|██████████████▌                                                             | 4974/26010 [00:16<01:19, 264.97it/s]


 19%|██████████████▌    

 28%|█████████████████████▏                                                      | 7240/26010 [00:23<00:57, 324.49it/s]


 28%|█████████████████████▎                                                      | 7273/26010 [00:23<00:58, 319.76it/s]


 28%|█████████████████████▎                                                      | 7306/26010 [00:23<01:11, 262.32it/s]


 28%|█████████████████████▍                                                      | 7337/26010 [00:23<01:08, 274.48it/s]


 28%|█████████████████████▌                                                      | 7366/26010 [00:23<01:07, 275.20it/s]


 28%|█████████████████████▋                                                      | 7401/26010 [00:23<01:04, 290.61it/s]


 29%|█████████████████████▊                                                      | 7445/26010 [00:23<00:57, 321.63it/s]


 29%|█████████████████████▊                                                      | 7483/26010 [00:23<00:55, 335.60it/s]


 29%|███████████████████

...

 70%|████████████████████████████████████████████████████▋                      | 18258/26010 [01:02<00:38, 201.39it/s]


 70%|████████████████████████████████████████████████████▋                      | 18281/26010 [01:02<00:37, 207.65it/s]


 70%|████████████████████████████████████████████████████▊                      | 18315/26010 [01:02<00:34, 223.22it/s]


 71%|████████████████████████████████████████████████████▉                      | 18349/26010 [01:02<00:31, 246.85it/s]


 71%|████████████████████████████████████████████████████▉                      | 18376/26010 [01:03<00:31, 242.00it/s]


 71%|█████████████████████████████████████████████████████                      | 18402/26010 [01:03<00:31, 243.85it/s]


 71%|█████████████████████████████████████████████████████▏                     | 18429/26010 [01:03<00:30, 245.83it/s]


 71%|█████████████████████████████████████████████████████▏                     | 18455/26010 [01:03<00:32, 235.21it/s]


 71%|███████████████████

 79%|██████████████████████████████████████████████████████████▉                | 20451/26010 [01:10<00:21, 262.60it/s]


 79%|███████████████████████████████████████████████████████████                | 20489/26010 [01:10<00:19, 282.51it/s]


 79%|███████████████████████████████████████████████████████████▏               | 20519/26010 [01:10<00:27, 199.96it/s]


 79%|███████████████████████████████████████████████████████████▏               | 20545/26010 [01:10<00:25, 213.93it/s]


 79%|███████████████████████████████████████████████████████████▎               | 20570/26010 [01:10<00:30, 180.31it/s]


 79%|███████████████████████████████████████████████████████████▍               | 20592/26010 [01:11<00:32, 164.69it/s]


 79%|███████████████████████████████████████████████████████████▍               | 20611/26010 [01:11<00:36, 146.69it/s]


 79%|███████████████████████████████████████████████████████████▍               | 20628/26010 [01:11<00:37, 142.00it/s]


 79%|███████████████████

 84%|███████████████████████████████████████████████████████████████▏           | 21930/26010 [01:19<00:17, 239.42it/s]


 84%|███████████████████████████████████████████████████████████████▎           | 21956/26010 [01:19<00:17, 236.74it/s]


 85%|███████████████████████████████████████████████████████████████▍           | 21988/26010 [01:19<00:15, 256.31it/s]


 85%|███████████████████████████████████████████████████████████████▍           | 22015/26010 [01:19<00:17, 229.96it/s]


 85%|███████████████████████████████████████████████████████████████▌           | 22044/26010 [01:19<00:16, 241.68it/s]


 85%|███████████████████████████████████████████████████████████████▋           | 22071/26010 [01:19<00:16, 244.97it/s]


 85%|███████████████████████████████████████████████████████████████▊           | 22112/26010 [01:19<00:14, 278.18it/s]


 85%|███████████████████████████████████████████████████████████████▊           | 22142/26010 [01:19<00:14, 266.45it/s]


 85%|███████████████████

 92%|█████████████████████████████████████████████████████████████████████▏     | 23986/26010 [01:27<00:09, 213.19it/s]


 92%|█████████████████████████████████████████████████████████████████████▏     | 24010/26010 [01:27<00:09, 202.37it/s]


 92%|█████████████████████████████████████████████████████████████████████▎     | 24033/26010 [01:27<00:09, 209.51it/s]


 92%|█████████████████████████████████████████████████████████████████████▎     | 24056/26010 [01:27<00:09, 213.02it/s]


 93%|█████████████████████████████████████████████████████████████████████▍     | 24081/26010 [01:27<00:08, 220.13it/s]


 93%|█████████████████████████████████████████████████████████████████████▌     | 24114/26010 [01:27<00:07, 243.10it/s]


 93%|█████████████████████████████████████████████████████████████████████▋     | 24160/26010 [01:27<00:06, 282.26it/s]


 93%|█████████████████████████████████████████████████████████████████████▊     | 24192/26010 [01:27<00:06, 273.37it/s]


 93%|███████████████████

Number of candidates: 8167
Wall time: 1min 36s


In [19]:
labeled = []
for c in session.query(Colleagues).all():
    labeled.append(c)
print("Number labeled:", len(labeled))

Number labeled: 8167


In [20]:
from snorkel.viewer import SentenceNgramViewer

SentenceNgramViewer(labeled, session)

<IPython.core.display.Javascript object>

SentenceNgramViewer(cids=[[[733], [5939], [7257]], [[1405], [4448, 4449, 5651], [7792, 7793, 7883]], [[6624, 6…